<a href="https://www.kaggle.com/code/averma111/pytorch-ps-s3e14?scriptVersionId=128235897" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


### Global Setting

In [ ]:
sns.set_style("darkgrid")
pd.set_option('mode.chained_assignment',None)

###  Assigning the directory and file paths

In [ ]:
ROOT_PATH='/kaggle/input/playground-series-s3e14'
train_file = 'train.csv'
test_file = 'test.csv'
sample = 'sample_submission.csv'

### Reading the train data

In [ ]:
train = pd.read_csv(ROOT_PATH+'/'+train_file)
train.head()

### Defining the summary function

In [ ]:
def summary(text, df):
    print(f'{text} shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summ['null'] = df.isnull().sum()
    summ['unique'] = df.nunique()
    summ['min'] = df.min()
    summ['median'] = df.median()
    summ['max'] = df.max()
    summ['mean'] = df.mean()
    summ['std'] = df.std()
    return summ

###  Summary of the trained data

In [ ]:
summary('train',train)

### Distribution of the target label 

In [ ]:
sns.displot(train['yield'])
## Distribution looks fairely normal